In [ ]:
### 檔案處理（多行轉成單行）
import openpyxl
import pandas as pd
import random

# 載入Excel檔案
file_path = '組織行為考題.xlsx'
excel_file = pd.ExcelFile(file_path)

# 遍歷每個儲存格，並將多行文字轉換為單行
for i, sheet_name in enumerate(excel_file.sheet_names):
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active
    for row in ws.iter_rows():
        for cell in row:
            if cell.value and isinstance(cell.value, str):
                # 將多行文字中的換行符號(\n)替換成空格
                cell.value = cell.value.replace('\n', ' ')

# 保存檔案
wb.save('處理後的檔案.xlsx')


In [ ]:
!pip install pandas googletrans==4.0.0-rc1

In [ ]:
!pip install --upgrade httpx

In [3]:
import pandas as pd
from googletrans import Translator

# 載入您的 Excel 文件
file_path = '處理後的檔案.xlsx'  # 將這裡替換為您的文件路徑
excel_file = pd.ExcelFile(file_path)

# 初始化翻譯器
translator = Translator()

# 用於儲存翻譯後的工作表
translated_sheets = {}

# 逐一處理每個工作表
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    
    # 先翻譯欄位名稱
    new_columns = []
    for col in df.columns:
        # 原始英文欄位名稱
        new_columns.append(col)
        # 翻譯成繁體中文的欄位名稱
        translated_col = translator.translate(col, src='en', dest='zh-tw').text
        new_columns.append(f"{translated_col}")

    # 準備儲存新的欄位數據
    translated_data = []

    for index, row in df.iterrows():
        row_data = []
        for text in row:
            # 原始英文內容
            row_data.append(text)
            # 翻譯成繁體中文的內容
            translation = translator.translate(str(text), src='en', dest='zh-tw')
            row_data.append(translation.text)
        
        translated_data.append(row_data)
    
    # 建立新 DataFrame，使用新欄位名稱
    translated_df = pd.DataFrame(translated_data, columns=new_columns)
    
    # 儲存結果到字典中
    translated_sheets[sheet_name] = translated_df

# 將更新後的工作表寫回到新的 Excel 文件
with pd.ExcelWriter('translated_excel_file.xlsx') as writer:
    for sheet_name, translated_df in translated_sheets.items():
        translated_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [2]:
import pandas as pd
import random
import string

# 載入Excel檔案
file_path = '處理後的檔案.xlsx'
excel_file = pd.ExcelFile(file_path)

# 建立一個包含所有資料的 DataFrame
all_data = pd.concat([pd.read_excel(file_path, sheet_name=sheet) for sheet in excel_file.sheet_names])

while True:
    # 列出所有工作表並加入 "ALL" 選項
    print("可選擇的資料表如下：")
    for i, sheet_name in enumerate(excel_file.sheet_names):
        print(f"{i + 1}. {sheet_name}")
    print(f"{len(excel_file.sheet_names) + 1}. ALL")

    # 讓使用者選擇資料表
    sheet_index = int(input("請輸入想測驗的資料表序號：")) - 1
    if sheet_index == len(excel_file.sheet_names):
        df = all_data.copy()  # 選擇 "ALL" 選項時合併所有資料表
    else:
        df = pd.read_excel(file_path, sheet_name=excel_file.sheet_names[sheet_index])

    # 初始化分數和錯誤清單
    score = 0
    wrong_answers = []

    # 隨機打亂資料
    df = df.sample(frac=1).reset_index(drop=True)

    # 迭代每一行資料進行測驗
    for index, row in df.iterrows():
        explanation = row[1]  # 第二欄為解釋
        correct_answer = row[0]  # 第一欄為正確答案
        
        # 隨機選取幾個干擾選項
        options = [correct_answer]
        while len(options) < 5:  # 總共提供5個選項
            distractor = df.sample(1).iloc[0, 0]  # 隨機取另一個單字
            if distractor != correct_answer and distractor not in options:
                options.append(distractor)
        
        # 打亂選項順序
        random.shuffle(options)

        # 顯示選項
        print("\n解釋：", explanation)
        choices = dict(zip(string.ascii_uppercase[:len(options)], options))
        for letter, word in choices.items():
            print(f"{letter}. {word}")

        # 輸入選項
        user_choice = input("請輸入對應的選項（A, B, C, D, ...）： ").strip().upper()
        if user_choice in choices and choices[user_choice] == correct_answer:
            score += 1
            print("答對了！\n")
        else:
            print(f"答錯了! 正確答案是: {correct_answer}\n")
            wrong_answers.append({
                "解釋": explanation,
                "正確答案": correct_answer,
                "你的答案": choices.get(user_choice, "無效的選擇")
            })

    # 顯示總分
    print(f"測驗結束! 你的總分是: {score}/{len(df)}")

    # 列出錯誤題目
    if wrong_answers:
        print("\n以下是你答錯的題目：")
        for wrong in wrong_answers:
            print(f"解釋: {wrong['解釋']}")
            print(f"正確答案: {wrong['正確答案']}")
            print(f"你的答案: {wrong['你的答案']}\n")
    else:
        print("恭喜你，全都答對了!")

    # 提供再次測驗或退出選項
    again = input("是否要再測驗一次？輸入 'Q' 結束，或按任意鍵繼續： ")
    if again.strip().upper() == "Q":
        print("測驗結束，感謝參加！")
        break


可選擇的資料表如下：
1. C1
2. C2
3. C3
4. C4
5. C5
6. C6
7. ALL
請輸入想測驗的資料表序號：1

解釋： A process that includes defining goals, establishing strategy, and developing plans to coordinate activities.
A. conceptual skills
B. sociology
C. social psychology
D. processes
E. planning
請輸入對應的選項（A, B, C, D, ...）： e
答對了！


解釋： The degree to which an organization meets the needs of its clientele or customers.
A. psychology
B. effectiveness
C. controlling
D. anthropology
E. people skills
請輸入對應的選項（A, B, C, D, ...）： B
答對了！


解釋： The ability to apply specialized knowledge or expertise.
A. social psychology
B. contingency variables
C. technical skills
D. withdrawal behavior
E. organization
請輸入對應的選項（A, B, C, D, ...）： c
答對了！


解釋： Discretionary behavior that contributes to the psychological and social environment of the workplace.
A. efficiency
B. organizational citizenship behavior (OCB)
C. processes
D. organization
E. contingency variables
請輸入對應的選項（A, B, C, D, ...）： B
答對了！


解釋： The set of actions employees take to 

請輸入對應的選項（A, B, C, D, ...）： a
答對了！


解釋： A national culture attribute that describes a tight social framework in which people expect others in groups of which they are a part to look after them and protect them.
A. collectivism
B. uncertainty avoidance
C. discrimination
D. individualism
E. masculinity
請輸入對應的選項（A, B, C, D, ...）： a
答對了！


解釋： When employment practices have an intentional discriminatory effect on a legally protected group of people
A. power distance
B. discrimination
C. disparate treatment
D. prejudice
E. masculinity
請輸入對應的選項（A, B, C, D, ...）： c
答對了！

測驗結束! 你的總分是: 9/12

以下是你答錯的題目：
解釋: Actions or behaviors that create, maintain, or reinforce some groups’ advantages over other groups and their members.
正確答案: discrimination
你的答案: 無效的選擇

解釋: Prejudice that may be hidden outside one’s conscious awareness.
正確答案: implicit bias
你的答案: collectivism

解釋: Differences in values, personality, and work preferences that become progressively more important for determining similarity as peo

請輸入對應的選項（A, B, C, D, ...）： 
答錯了! 正確答案是: deep acting


解釋： Intense, discrete, and shortlived feeling experiences, often caused by a specific event.
A. surface acting
B. affective events theory (AET)
C. positive affect
D. emotions
E. positivity offset


KeyboardInterrupt: Interrupted by user